
# Advent of Code 2023

> The effort of using machines to mimic the human mind has always struck me as rather silly. I would rather use them to mimic something better.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
#type: ignore
from functools import reduce
from itertools import product, accumulate
from operator import mul, or_
from math import gcd
from dataclasses import dataclass
from collections import Counter, defaultdict
import re


def prod(u): 
    """Older Pythons don't have this."""
    return reduce(mul, u)


def lcm(*u):
    """Older Pythons don't have this."""
    return reduce(lambda a, b: a * b // gcd(a, b), u)


def inputfunc(day, kind='lines', testing=False):
    """Generator to read input files."""
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'commas':
            text = [x.strip() for x in open(filename).read().split(',')]
        elif kind == 'raw':
            text = open(filename).read()

        def inner():
            return func(f=text)
        return inner
    return gen

## [Day 1 - Trebuchet?!](https://adventofcode.com/2023/day/1)

In [2]:
@inputfunc(1)
def input_1(f):
    return f


def find_digits(s):
    return [int(i) for i in s if i.isdigit()]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digits(s) for s in input_1())
)
assert A == 55123


d = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

r = re.compile(f'(?=(\\d|{"|".join(d)}))')

def find_digit_names(s):
    def to_number(p):
        if p.isdigit():
            return int(p)
        else:
            return d[p]

    return [to_number(p) for p in r.findall(s)]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digit_names(s) for s in input_1())
)
assert A == 55260

## [Day 2 - Cube Conundrum](https://adventofcode.com/2023/day/2)

In [3]:
@dataclass
class Game:
    gameid: int
    contents: list[Counter[str, int]]

        
@inputfunc(2)
def input_2(f):
    for l in f:
        num, items = l.split(':')
        
        _, gameid = num.split()
        
        contents = [
            Counter({
                b: int(a)
                for a, b in [
                    y.split()
                    for y in x.split(',')
                ]
            })
            for x in items.split(';')
        ]
        
        yield Game(int(gameid), contents)


games = list(input_2())


cubes = Counter({'red': 12, 'green': 13, 'blue': 14})
A = sum(
    game.gameid 
    for game in games
    if all(
        d < cubes for d in game.contents
    )
)
assert A == 2476


A = sum(
    prod(
        reduce(or_, game.contents).values()
    )
    for game in games
)
assert A == 54911

## [Day 3 - Gear Ratios](https://adventofcode.com/2023/day/3)

In [4]:
class Grid:
    def __init__(self, g):
        self._grid = g
        self.columns = len(g[0])
        self.rows = len(g)

    def __getitem__(self, t):
        x, y = t
        return self._grid[y][x]
    
    def adjacent(self, x, y):
        return [
            self[i, j] 
            for i, j in {
                (x+1, y), (x-1, y), (x, y+1), (x, y-1),
                (x+1, y+1), (x-1, y+1), (x+1, y-1), (x-1, y-1)
            }
            if 0 <= i < self.columns and 0 <= j < self.rows
        ]
    

@dataclass
class PartNumber:
    value: int
    col_start: int
    col_end: int
    row: int


@inputfunc(3)
def input_3(f):
    return Grid([
        list(x) for x in f
    ])
    

def find_part_numbers(grid, is_symbol):
    val, symbol_adjacent = 0, False

    for j in range(grid.rows):
        for i in range(grid.columns):
            c = grid[i, j]
            if c.isdigit():
                val = 10 * val + int(grid[i, j])
            if not c.isdigit():
                if val and symbol_adjacent:
                    yield PartNumber(
                        val, i - len(str(val)), i - 1, j
                    )
                val, symbol_adjacent = 0, False
            if val and any(is_symbol(s) for s in grid.adjacent(i, j)):
                symbol_adjacent = True
        if val and symbol_adjacent:
            yield PartNumber(
                val, grid.columns - len(str(val)), grid.columns - 1, j
            )
        val, symbol_adjacent = 0, False
            
            
A = sum(
    p.value for p in find_part_numbers(
        input_3(), 
        lambda c: not (c.isdigit() or c == '.')
    )
)
assert A == 536576


def find_gear_ratios(grid):
    part_numbers = defaultdict(list)
    for p in find_part_numbers(grid, lambda c: c == '*'):
        part_numbers[p.row].append(p)
    
    for i, j in product(range(grid.columns), range(grid.rows)):
        if grid[i, j] != '*':
            continue
                
        adjacents = (
            [
                p for p in part_numbers[j] 
                if p.col_start == i+1 or p.col_end == i-1
            ] + [
                p for p in part_numbers[j-1] + part_numbers[j+1]
                if p.col_start <= i+1 and p.col_end >= i-1
            ]
        )
            
        if len(adjacents) == 2:
            yield adjacents
            
    
A = sum(a.value * b.value for a, b in find_gear_ratios(input_3()))
assert A == 75741499

## [Day 4 - Scratchcards](https://adventofcode.com/2023/day/4)

Part 1 is very simple. The only observation is that it's one of the many declinations of the count distinct problem: let the sets of winning numbers and of the player's numbers be $M$ and $N$ respectively, then $|M \cap N| = |M| + |N| - |M \cup N|$. This has a worst-case lower bound of $\Theta(n \log n)$ [(for proof see: Grigoriev 99)](https://www.semanticscholar.org/paper/Complexity-lower-bounds-for-randomized-computation-Grigoriev/7b0c914a951bc59bb0cee33f197e6418a3c2600b), which may be achieved in a variety of ways.

The code implements the expected linear time randomized solution of merging two hash-sets (technically, its worst-case complexity is $\Theta(n^2)$, but that happens with probability 0).

Part 2 is also very simple. Let $n$ be the number of cards, and let $m$ be the maximum number of numbers appearing in any given card.

Winning cards generate more cards with higher numbers, and the hypotheses of the problem ensure that only copies of cards in the initial range are generated. This is sufficient to easily prove termination, and it suggests a naive algorithm: 

- initialize an array $A[1..n]$ with $A[i] = 1$ for all $1 \le i \le n$, where $A[i]$ represents the copies of card $i$ we have, initially one.
- for all $1 \le j \le n$, compute how many winning numbers $w_i$ the $i$-th card has, and for all $j+1 \le k \le j+w$, let $A[k] \leftarrow A[k] + A[j]$
- the final answer is $\sum A[i]$.

If we let $f(m)$ be the complexity of the subroutine that computes how many numbers a given card has, it's easy to see that the above algorithm has complexity bounded by $\Theta(n(m + f(m)))$. But because $f(m) \in \Omega(m)$, this is already optimal.

Bonus variant. Imagine that instead of having to compute the matching numbers on the cards, we are instead given an oracle for that, in other words, $f(m) \in \Theta(1)$. Can we do better than $\Theta(mn)$? We sure can, thanks to the prefix trick!

Let us initialize an array $A[1..n+1]$ with $A[i] = 0$ for all $2 \le i \le n$, $A[1] = 1$ and $A[n+1] = -1$.

The idea is that the number of copies of card $i$ we have is represented by $\sum_{j=0}^i A[j]$. If we wish to increase by $h$ the amount of copies of cards $r+1, r+2, \dots, r+w$, it is sufficient to let $A[r+1] \leftarrow A[r+1] + h$ and $A[r+w+1] \leftarrow A[r+w+1] - h$.

Therefore, for each $1 \le j \le n$, we let $w_j$ be the winning numbers of the $j$-th card, and $c_j$ be the number of copies of that card. We may simply set $A[j+1] \leftarrow A[j+1] + c_j$ and $A[j+w_j+1] \leftarrow A[r+w_j+1] - c_j$.

At each loop iteration, $c_j$ may be computed as $c_{j-1} + A[j]$ by definition.

The final answer will be:

$$
      \sum_{i=1}^{n} \sum_{j=1}^{i} A[i]
    = \sum_{i=1}^{n+1} \left( \sum_{j=1}^{n+1} A[j] - \sum_{j=i+1}^{n+1} A[j] \right)
    = (n+1) \left( \sum_{i=1}^{n+1} A[i] \right) -  \left( \sum_{i=1}^{n+1} \sum_{j=i}^{n+1} A[i] \right)
    = \sum_{i=1}^{n+1} (n-i+1) A[i]
$$

The initialization and final computation are linear-time operations, and each loop iteration is constant time, which yields an algorithm of complexity $\Theta(nf(m))$.

This could have been such a beautiful problem :-(


In [5]:
@dataclass
class Scratchcard:
    cardid: int
    winning: set[int]
    my: set[int]
    
    def winning_numbers(self):
        return len(self.winning & self.my)

    def score(self):
        s = self.winning_numbers()
        return s and 2 ** (s-1)


@inputfunc(4)
def input_4(f):
    for l in f:
        num, items = l.split(':')
        
        _, cardid = num.split()
        
        winning, my = (
            {int(y) for y in x.split()}
            for x in items.split('|')
        ) 
        
        yield Scratchcard(int(cardid), winning, my)
       
        
A = sum(c.score() for c in input_4())
assert A == 23941


def recursive_scratchcards(cards):
    cards_array = [c.winning_numbers() for c in cards]
    prefix_array = [1] + [0] * (len(cards_array) - 1) + [-1]
    for n, (w, c) in enumerate(zip(cards_array, accumulate(prefix_array))):
        prefix_array[n+1] += c
        prefix_array[n+w+1] -= c
    return sum(n * c for n, c in enumerate(reversed(prefix_array)))
    

A = recursive_scratchcards(input_4())
assert A == 5571760